In [2]:
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

**Data Souce Table**

| Database | ESG Evaluation Form | Time Range | Data Structure |
| --- | --- | --- | --- |
| CNRDS | Float score | 2007-2022 | Organized Dataframe |
| MSCI | Float score | 2006-2020 | *unorganized Dataframe* |
| Wind 万得 | Float score | 2018-2023 | Organized Dataframe |
| HUAZHENG 华政 | Float score | 2009-2023 | Organized Dataframe |
| SynTao 商道融绿 | Discrete ratings | 2015-2023 | Organized Dataframe |
| FTSE Russel 富时罗素 | Float score | 2015-2023 | Organized Dataframe |
| Bloomberg 彭博 | Float score | 2006-2022 | Organized Dataframe |
| RUNLING 润灵环球 | Float score + Discrete ratings | 2019-2023 | Organized Dataframe |
| Allied Wave 盟浪 | Discrete ratings | 2014-2023 | Organized Dataframe |
| HEXUN 和讯 | Float score + Discrete ratings | 2010-2020 | Organized Dataframe |

注意：华证使用的是一年中月度ESG数据的均值&中位数，其余均为年度评级

共计10个数据源，时间跨度为2006-2023

数据清洗后格式：
- 键：ts_code, name, year
- ESG指标：esg_rank_norm_{source}
- 其他：industry, 原始ESG分数等

基本column: 
```python
col = ['ts_code', 'name', 'year', 'esg_score', 'esg_score_rank', 'esg_rank_norm_{name}']
```

**行业分类采用WIND & 证监会（CRSC）标准**

---

**处理方法**：
1. 将不同的ESG评级转变为排名，并将其归一化至[0, 1]区间
2. 数据源之间两两配对，计算标准差作为该配对的分歧 i.e. 极差绝对值除以根号2 $\frac{|r_{i,t,1}-r_{i,t,2}|}{\sqrt{2}}$
3. 对每个公司每年的所有分歧取均值，作为该公司该年的ESG分歧指标

# CNRDS

In [2]:
CNRDS_data_path = Path('ESG_Data\CNRDS esg评级\CNRDSesg评级已匹配.xlsx')
CNRDS_data = pd.read_excel(CNRDS_data_path, sheet_name='Sheet1')
CNRDS_data.head()

,股票代码,公司简称,会计年度,ESG得分,ESG排名,E得分,E排名,S得分,S排名,G得分,...,制造业为1否则为0,沪深A股为1否则为0,第一种重污染行业为1否则为0,第二种重污染行业为1否则为0,第三种重污染行业为1否则为0,产权性质,所属省份代码,所属城市代码,所在省份,所在地级市
0,1,平安银行,2007,16.2574,854,4.3057,453,6.9942,1101,42.5014,...,0,1,0,0,0,0.0,440000.0,440300.0,广东省,深圳市
1,1,平安银行,2008,15.4370,1058,5.1238,594,14.2385,733,20.4718,...,0,1,0,0,0,0.0,440000.0,440300.0,广东省,深圳市
2,1,平安银行,2009,14.8069,1197,1.4407,1527,20.0804,523,19.5958,...,0,1,0,0,0,0.0,440000.0,440300.0,广东省,深圳市
3,1,平安银行,2010,22.0068,538,4.9378,829,18.5075,857,43.2817,...,0,1,0,0,0,0.0,440000.0,440300.0,广东省,深圳市
4,1,平安银行,2011,17.4579,1152,1.4384,2035,17.3651,1096,32.7329,...,0,1,0,0,0,0.0,440000.0,440300.0,广东省,深圳市


In [3]:
year_lst = CNRDS_data['会计年度'].unique()

CNRDS_data_trans = pd.DataFrame(columns=['股票代码', '公司简称', '会计年度', 'ESG得分', 'ESG排名', 'ESGRank'])

for year in tqdm(year_lst):
    data_year = CNRDS_data[CNRDS_data['会计年度'] == year].sort_values(by='股票代码').reset_index(drop=True)
    data_year_select = data_year[['股票代码', '公司简称', '会计年度', 'ESG得分', 'ESG排名']]
    data_year_select['ESGRank'] = data_year_select['ESG排名'] / data_year_select['ESG排名'].max()
    CNRDS_data_trans = pd.concat([
        CNRDS_data_trans, data_year_select
    ])

CNRDS_data_trans.reset_index(drop=True, inplace=True)
CNRDS_data_trans

  0%|          | 0/16 [00:00<?, ?it/s]

,股票代码,公司简称,会计年度,ESG得分,ESG排名,ESGRank
0,1,平安银行,2007,16.2574,854,0.578591
1,2,万科A,2007,28.1900,184,0.124661
2,4,ST国华,2007,16.8875,790,0.535230
3,5,ST星源,2007,12.0990,1209,0.819106
4,6,深振业A,2007,17.2138,753,0.510163
...,...,...,...,...,...,...
46815,873169,七丰精工,2022,17.9424,4583,0.895992
46816,873223,荣亿精密,2022,22.8906,3615,0.706745
46817,873305,九菱科技,2022,13.5136,4998,0.977126
46818,873339,恒太照明,2022,17.1917,4677,0.914370


In [4]:
CNRDS_data_trans.columns = [
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank', 'esg_rank_norm_CNRDS'
]
CNRDS_data_trans_path = Path("ESG_Data_transformed\CNRDS_esg_rating_transformed.csv")
CNRDS_data_trans.to_csv(CNRDS_data_trans_path, index=False, encoding="utf_8_sig")

# MSCI

In [5]:
MSCI_data_path = Path('ESG_Data\MSCI-ESG评分2010-2020年数据\A股MSCI+ESG-Cleaned.xlsx')
MSCI_data_raw = pd.read_excel(MSCI_data_path, sheet_name='Current Screen Template')
MSCI_data_raw.head()

,Identifier,Company Name,Country of Exchange,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
0,300076.SZ,Ningbo GQY Video & Telecom Joint-stock Co Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,900957.SS,Shanghai Lingyun Industries Development Co Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,900953.SS,Kama Co Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,900948.SS,Inner Mongolia Yitai Coal Co Ltd,China,76.959725,62.449852,55.900696,66.993813,41.255987,20.99021,20.254328,18.715875,23.739281,13.690186,8.078445,NaN,NaN,NaN,NaN
4,900939.SS,Shanghai Huili Building Materials Co Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
MSCI_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score', 'esg_score_rank'
])

for year in tqdm(range(2006, 2021)):
    data_year = MSCI_data_raw[["Identifier", "Company Name", year]].reset_index(drop=True)
    data_year['year'] = year
    data_year.columns = ['ts_code', 'name', 'esg_score', 'year']

    data_year['esg_score_rank'] = 1 - data_year['esg_score'].rank() / data_year['esg_score'].rank().max()

    MSCI_data = pd.concat([
        MSCI_data, data_year[['ts_code', 'name', 'year', 'esg_score', 'esg_score_rank']]
    ])

MSCI_data.reset_index(drop=True, inplace=True)
MSCI_data.head()

  0%|          | 0/15 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_score_rank
0,300076.SZ,Ningbo GQY Video & Telecom Joint-stock Co Ltd,2006,NaN,NaN
1,900957.SS,Shanghai Lingyun Industries Development Co Ltd,2006,NaN,NaN
2,900953.SS,Kama Co Ltd,2006,NaN,NaN
3,900948.SS,Inner Mongolia Yitai Coal Co Ltd,2006,NaN,NaN
4,900939.SS,Shanghai Huili Building Materials Co Ltd,2006,NaN,NaN


In [7]:
MSCI_data.sort_values(by=['ts_code', 'year'], inplace=True)
MSCI_data.reset_index(drop=True, inplace=True)

In [8]:
MSCI_data

,ts_code,name,year,esg_score,esg_score_rank
0,000001.SZ,Ping An Bank Co Ltd,2006,NaN,NaN
1,000001.SZ,Ping An Bank Co Ltd,2007,NaN,NaN
2,000001.SZ,Ping An Bank Co Ltd,2008,NaN,NaN
3,000001.SZ,Ping An Bank Co Ltd,2009,NaN,NaN
4,000001.SZ,Ping An Bank Co Ltd,2010,NaN,NaN
...,...,...,...,...,...
71440,900957.SS,Shanghai Lingyun Industries Development Co Ltd,2016,NaN,NaN
71441,900957.SS,Shanghai Lingyun Industries Development Co Ltd,2017,NaN,NaN
71442,900957.SS,Shanghai Lingyun Industries Development Co Ltd,2018,NaN,NaN
71443,900957.SS,Shanghai Lingyun Industries Development Co Ltd,2019,NaN,NaN


In [9]:
MSCI_data.columns = [
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_MSCI'
]
MSCI_data_path = Path("ESG_Data_transformed\MSCI_esg_rating_transformed.csv")
MSCI_data.to_csv(MSCI_data_path, index=False, encoding="utf_8_sig")

# WIND

In [10]:
WIND_data_path = Path('ESG_Data\windESG评级（19-23年）\Wind ESG评级2018-2023.xlsx')
wind_data_raw = pd.read_excel(WIND_data_path, sheet_name='Sheet1')
wind_data_raw.head()

,股票代码,年份,公司名称,评级日期,WindESG评级,WindESG综合得分,管理实践得分,争议事件得分,环境得分,社会得分,治理得分,证监会行业,Wind行业
0,1,2018.0,平安银行,2018-03-22,B,5.03,2.86,2.17,2.53,1.75,7.45,货币金融服务,商业银行
1,1,2019.0,平安银行,2019-03-14,BB,5.86,4.06,1.80,3.16,5.35,7.24,货币金融服务,商业银行
2,1,2020.0,平安银行,2020-02-21,BBB,5.82,3.69,2.13,2.79,3.81,7.87,货币金融服务,商业银行
3,1,2021.0,平安银行,2021-02-09,A,6.85,5.10,1.74,5.02,7.48,7.81,货币金融服务,商业银行
4,1,2022.0,平安银行,2022-03-17,A,7.67,5.38,2.29,7.06,8.03,7.46,货币金融服务,商业银行


In [11]:
wind_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank', 'esg_rank_norm_WIND', 'CSRC_industry', 'WIND_industry'
])

for year in tqdm(range(2018, 2024)):
    data_year = wind_data_raw[wind_data_raw['年份'] == year].reset_index(drop=True)
    data_year_select = data_year[["股票代码", "公司名称", "年份", "WindESG综合得分", "WindESG评级", "证监会行业","Wind行业"]]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_score', 'esg_rank', 'CSRC_industry', 'WIND_industry']
    data_year_select['esg_rank_norm_WIND'] = 1- data_year_select['esg_score'].rank() / data_year_select['esg_score'].rank().max()
    data_year_select['year'] = year
    wind_data = pd.concat([
        wind_data, 
        data_year_select[['ts_code', 'name', 'year', 'esg_score', 'esg_rank', 'esg_rank_norm_WIND', 'CSRC_industry', 'WIND_industry']]
    ])

wind_data.sort_values(by=['ts_code', 'year'], inplace=True)
wind_data.reset_index(drop=True, inplace=True)
wind_data

  0%|          | 0/6 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_rank,esg_rank_norm_WIND,CSRC_industry,WIND_industry
0,1,平安银行,2018,5.03,B,0.866542,货币金融服务,商业银行
1,1,平安银行,2019,5.86,BB,0.479126,货币金融服务,商业银行
2,1,平安银行,2020,5.82,BBB,0.519621,货币金融服务,商业银行
3,1,平安银行,2021,6.85,A,0.155494,货币金融服务,商业银行
4,1,平安银行,2022,7.67,A,0.037926,货币金融服务,商业银行
...,...,...,...,...,...,...,...,...
25419,873223,荣亿精密,2022,4.79,B,0.959061,通用设备制造业,机械
25420,873223,荣亿精密,2023,5.86,BB,0.520180,通用设备制造业,机械
25421,873305,九菱科技,2023,5.16,BB,0.878723,金属制品业,金属、非金属与采矿
25422,873339,恒太照明,2023,5.40,BB,0.776058,电气机械和器材制造业,家庭耐用消费品


In [12]:
WIND_trans_data_path = Path("ESG_Data_transformed\WIND_esg_rating_transformed.csv")
wind_data.to_csv(WIND_trans_data_path, index=False, encoding="utf_8_sig")

# HUAZHENG

排序使用均值进行归一化

In [13]:
huazheng_data_path = Path('ESG_Data\华证2009-2023年（含细分项+季度)）\华证ESG评级_clean.xlsx')
huazheng_data_raw = pd.read_excel(huazheng_data_path, sheet_name='Sheet1')
huazheng_data_raw

,股票代码,证券简称,年份,ESG得分-年均值,ESG得分-年中位数,Unnamed: 5
0,1,平安银行,2009,6.000000,6.0,NaN
1,1,平安银行,2010,6.000000,6.0,NaN
2,1,平安银行,2011,6.000000,6.0,NaN
3,1,平安银行,2012,6.000000,6.0,NaN
4,1,平安银行,2013,6.500000,6.5,NaN
...,...,...,...,...,...,...
47524,873576,天力复合,2023,3.500000,3.5,NaN
47525,873593,鼎智科技,2023,4.333333,4.0,NaN
47526,873665,科强股份,2023,4.000000,4.0,NaN
47527,873693,阿为特,2023,4.000000,4.0,NaN


In [14]:
huazheng_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score_mean', 'esg_score_median', 'esg_rank_norm_huazheng'
])

for year in tqdm(range(2009, 2024)):
    data_year = huazheng_data_raw[huazheng_data_raw['年份'] == year].reset_index(drop=True)
    data_year_select = data_year[["股票代码", "证券简称", "年份", "ESG得分-年均值", "ESG得分-年中位数"]]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_score_mean', 'esg_score_median']
    data_year_select['esg_rank_norm_huazheng'] = 1- data_year_select['esg_score_mean'].rank() / data_year_select['esg_score_mean'].rank().max()

    huazheng_data = pd.concat([
        huazheng_data, 
        data_year_select[['ts_code', 'name', 'year', 'esg_score_mean', 'esg_score_median', 'esg_rank_norm_huazheng']]
    ])

huazheng_data.sort_values(by=['ts_code', 'year'], inplace=True)
huazheng_data.reset_index(drop=True, inplace=True)
huazheng_data

  0%|          | 0/15 [00:00<?, ?it/s]

,ts_code,name,year,esg_score_mean,esg_score_median,esg_rank_norm_huazheng
0,1,平安银行,2009,6.000000,6.0,0.007687
1,1,平安银行,2010,6.000000,6.0,0.009353
2,1,平安银行,2011,6.000000,6.0,0.010138
3,1,平安银行,2012,6.000000,6.0,0.020365
4,1,平安银行,2013,6.500000,6.5,0.003603
...,...,...,...,...,...,...
47524,873576,天力复合,2023,3.500000,3.5,0.860446
47525,873593,鼎智科技,2023,4.333333,4.0,0.386840
47526,873665,科强股份,2023,4.000000,4.0,0.620367
47527,873693,阿为特,2023,4.000000,4.0,0.620367


In [15]:
huazheng_data_trans_path = Path("ESG_Data_transformed\huazheng_esg_rating_transformed.csv")
huazheng_data.to_csv(huazheng_data_trans_path, index=False, encoding="utf_8_sig")

# 商道融绿

In [16]:
syntao_data_path = Path("ESG_Data\商道融绿ESG评级数据2015-2023年\商道融绿esg评级（2015-2023）.xlsx")
syntao_data_raw = pd.read_excel(syntao_data_path, sheet_name='Sheet1')
syntao_data_raw

,stkcd,year,证券代码,代码,公司名称,评级日期,ESG综合评级,历史评级,总市值亿元CNY,流通市值亿元CNY,市盈率PETTM,每股收益EPSTTM,每股营业收入TTM,每股经营活动产生的现金流量净额TTM,证监会行业,Wind行业
0,1,2015,1,000001.SZ,平安银行,2015-06-30,B-,NaN,1500.980127,1238.245325,7.041896,1.489656,5.957854,10.791984,货币金融服务,银行
1,1,2016,1,000001.SZ,平安银行,2016-06-30,B-,"20150630, B-",1543.619982,1315.343117,6.915240,1.300027,6.000089,2.675941,货币金融服务,银行
2,1,2017,1,000001.SZ,平安银行,2017-06-30,B,"20160630, B-\n20150630, B-",1890.462291,1862.670661,8.318134,1.323614,6.283775,-9.509790,货币金融服务,银行
3,1,2018,1,000001.SZ,平安银行,2018-06-30,B,"20170630, B\n20160630, B-\n20150630, B-",1535.034776,1535.020062,6.512663,1.372710,6.179235,2.193890,货币金融服务,银行
4,1,2019,1,000001.SZ,平安银行,2019-06-30,B+,"20180630, B\n20170630, B\n20160630, B-\n201506...",2431.330249,2431.306812,9.055571,1.563678,7.414150,-2.233260,货币金融服务,银行
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5741,688819,2023,688819,688819.SH,天能股份,2023-03-19,B,"20221218, B\n20220918, B\n20220619, B",358.121640,358.121640,17.153891,2.147618,45.381823,4.810185,电气机械和器材制造业,资本货物
5742,688981,2022,688981,688981.SH,中芯国际,2022-06-19,A-,"20200529, B+",3569.299503,875.603737,28.453341,1.587511,5.086442,3.527932,计算机、通信和其他电子设备制造业,半导体与半导体生产设备
5743,688981,2023,688981,688981.SH,中芯国际,2023-03-19,A-,"20221218, A-\n20220918, A-\n20220619, A-\n2020...",4003.761164,988.879854,36.796055,1.372973,6.040424,3.971882,计算机、通信和其他电子设备制造业,半导体与半导体生产设备
5744,689009,2022,689009,689009.SH,九号公司,2022-06-19,B-,NaN,316.619418,316.619418,74.738251,5.954113,130.495292,-1.955097,计算机、通信和其他电子设备制造业,耐用消费品与服装


In [17]:
syntao_data_raw['ESG综合评级'].unique()

array(['B-', 'B', 'B+', 'A-', 'C+', 'C', 'A'], dtype=object)

In [18]:
syntao_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_rank', 'esg_rank_norm_syntao'
])

for year in tqdm(range(2015, 2024)):
    data_year = syntao_data_raw[syntao_data_raw['year'] == year].reset_index(drop=True)
    data_year_select = data_year[["stkcd", "公司名称", "year", "ESG综合评级"]]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_rank']
    data_year_select['esg_rank'] = data_year_select['esg_rank'].map(
        {
            "A":1, "A-":2, "B+":3, "B":4, "B-":5, "C+":6, "C":7
        }
    )
    data_year_select['esg_rank_norm_syntao'] = data_year_select['esg_rank'].rank() / data_year_select['esg_rank'].rank().max()

    syntao_data = pd.concat([
        syntao_data, 
        data_year_select[['ts_code', 'name', 'year', 'esg_rank', 'esg_rank_norm_syntao']]
    ])

syntao_data.sort_values(by=['ts_code', 'year'], inplace=True)
syntao_data.reset_index(drop=True, inplace=True)
syntao_data

  0%|          | 0/9 [00:00<?, ?it/s]

,ts_code,name,year,esg_rank,esg_rank_norm_syntao
0,1,平安银行,2015,5,0.457143
1,1,平安银行,2016,5,0.453782
2,1,平安银行,2017,4,0.198333
3,1,平安银行,2018,4,0.165486
4,1,平安银行,2019,3,0.066919
...,...,...,...,...,...
5741,688819,天能股份,2023,4,0.578156
5742,688981,中芯国际,2022,2,0.058271
5743,688981,中芯国际,2023,2,0.077175
5744,689009,九号公司,2022,5,0.786967


In [19]:
syntao_data_trans_path = Path("ESG_Data_transformed\syntao_esg_rating_transformed.csv")
syntao_data.to_csv(syntao_data_trans_path, index=False, encoding="utf_8_sig")

# FTSE Russel

In [20]:
ftse_data_path = Path("ESG_Data\富时罗素\富时罗素2018-2023（原始）.xlsx")
ftse_data_raw = pd.read_excel(ftse_data_path, sheet_name='万得')
ftse_data_raw

,证券代码,证券简称,富时罗素ESG评分\n[交易日期] 2023-12-31,富时罗素ESG评分\n[交易日期] 2022-12-31,富时罗素ESG评分\n[交易日期] 2021-12-31,富时罗素ESG评分\n[交易日期] 2020-12-31,富时罗素ESG评分\n[交易日期] 2019-12-31,富时罗素ESG评分\n[交易日期] 2018-12-31↓
0,000725.SZ,京东方A,2.8,2.8,2.5,2.5,2.5,2.2
1,600835.SH,上海机电,1.9,1.9,1.8,1.5,1.5,1.9
2,000488.SZ,晨鸣纸业,1.9,1.5,1.9,1.9,2.1,1.9
3,000550.SZ,江铃汽车,1.1,1.4,1.1,1.4,1.4,1.6
4,600618.SH,氯碱化工,1.4,1.4,1.4,1.4,1.4,1.3
...,...,...,...,...,...,...,...,...
5331,873679.BJ,前进科技,NaN,NaN,NaN,NaN,NaN,NaN
5332,873693.BJ,阿为特,NaN,NaN,NaN,NaN,NaN,NaN
5333,873703.BJ,广厦环能,NaN,NaN,NaN,NaN,NaN,NaN
5334,873726.BJ,卓兆点胶,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ftse_data_raw.columns = [
    'ts_code', 'name', '2023', '2022', '2021', '2020', "2019", "2018"
]
ftse_data_raw

,ts_code,name,2023,2022,2021,2020,2019,2018
0,000725.SZ,京东方A,2.8,2.8,2.5,2.5,2.5,2.2
1,600835.SH,上海机电,1.9,1.9,1.8,1.5,1.5,1.9
2,000488.SZ,晨鸣纸业,1.9,1.5,1.9,1.9,2.1,1.9
3,000550.SZ,江铃汽车,1.1,1.4,1.1,1.4,1.4,1.6
4,600618.SH,氯碱化工,1.4,1.4,1.4,1.4,1.4,1.3
...,...,...,...,...,...,...,...,...
5331,873679.BJ,前进科技,NaN,NaN,NaN,NaN,NaN,NaN
5332,873693.BJ,阿为特,NaN,NaN,NaN,NaN,NaN,NaN
5333,873703.BJ,广厦环能,NaN,NaN,NaN,NaN,NaN,NaN
5334,873726.BJ,卓兆点胶,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ftse_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_ftse'
])

for year in tqdm(range(2018, 2024)):
    data_year = ftse_data_raw[['ts_code', 'name', str(year)]].reset_index(drop=True)
    data_year['year'] = year
    data_year.columns = ['ts_code', 'name', 'esg_score', 'year']
    data_year['esg_rank_norm_ftse'] = 1 - data_year['esg_score'].rank() / data_year['esg_score'].rank().max()

    ftse_data = pd.concat([
        ftse_data, 
        data_year[['ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_ftse']]
    ])

ftse_data.sort_values(by=['ts_code', 'year'], inplace=True)
ftse_data.reset_index(drop=True, inplace=True)
ftse_data

  0%|          | 0/6 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_rank_norm_ftse
0,000001.SZ,平安银行,2018,NaN,NaN
1,000001.SZ,平安银行,2019,2.5,0.030461
2,000001.SZ,平安银行,2020,2.5,0.038621
3,000001.SZ,平安银行,2021,1.9,0.164096
4,000001.SZ,平安银行,2022,2.2,0.129104
...,...,...,...,...,...
32011,873833.BJ,美心翼申,2019,NaN,NaN
32012,873833.BJ,美心翼申,2020,NaN,NaN
32013,873833.BJ,美心翼申,2021,NaN,NaN
32014,873833.BJ,美心翼申,2022,NaN,NaN


In [23]:
ftse_data_trans_path = Path(r"ESG_Data_transformed\FTSE_esg_rating_transformed.csv")
ftse_data.to_csv(ftse_data_trans_path, index=False, encoding="utf_8_sig")

# Bloomberg

In [24]:
bloomberg_data_path = Path("ESG_Data\彭博ESG2006-2022\BloombergESG（匹配版）.xls")
bloomberg_data_raw = pd.read_excel(bloomberg_data_path, sheet_name='Sheet1')
bloomberg_data_raw

,Stkcd,Year,BloombergESG,BloombergE,BloombergS,BloombergG,ShortName,STPT,金融业,资不抵债,...,CITYCODE,CITY,East,West,Mid,HighTech_1,HighTech_2,Pollute_1,Pollute_2,Pollute_3
0,1,2007,11.6688,0.0000,0.0000,34.9187,深发展A,0.0,1.0,0.0,...,440300.0,深圳市,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2008,18.6903,7.9130,13.1802,34.9187,深发展A,0.0,1.0,0.0,...,440300.0,深圳市,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2009,20.8732,7.9130,17.3519,37.2968,深发展A,0.0,1.0,0.0,...,440300.0,深圳市,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2010,21.6678,7.9130,17.3519,39.6749,深发展A,0.0,1.0,0.0,...,440300.0,深圳市,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2011,21.0945,6.1915,17.3519,39.6749,深发展A,0.0,1.0,0.0,...,440300.0,深圳市,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16118,688981,2017,53.7370,46.9043,30.5925,83.5942,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16119,688981,2018,53.7370,46.9043,30.5925,83.5942,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16120,688981,2019,54.7933,46.9043,33.7666,83.5942,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16121,688981,2020,54.7933,46.9043,33.7666,83.5942,中芯国际,0.0,0.0,0.0,...,NaN,开曼群岛,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0


In [25]:
bloomberg_data = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_bloomberg'
])

for year in tqdm(range(2006, 2023)):
    data_year = bloomberg_data_raw[bloomberg_data_raw['Year'] == year].reset_index(drop=True)
    data_year_select = data_year[["Stkcd", "ShortName", "Year", "BloombergESG"]]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_score']
    data_year_select['esg_rank_norm_bloomberg'] = 1 - data_year_select['esg_score'].rank() / data_year_select['esg_score'].rank().max()
    
    bloomberg_data = pd.concat([
        bloomberg_data, 
        data_year_select[['ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_bloomberg']]
    ])

bloomberg_data.sort_values(by=['ts_code', 'year'], inplace=True)
bloomberg_data.reset_index(drop=True, inplace=True)
bloomberg_data

  0%|          | 0/17 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_rank_norm_bloomberg
0,1,深发展A,2007,11.6688,0.801237
1,1,深发展A,2008,18.6903,0.342187
2,1,深发展A,2009,20.8732,0.281471
3,1,深发展A,2010,21.6678,0.275000
4,1,深发展A,2011,21.0945,0.401639
...,...,...,...,...,...
16118,688981,NaN,2017,53.7370,0.014815
16119,688981,NaN,2018,53.7370,0.020867
16120,688981,NaN,2019,54.7933,0.024590
16121,688981,中芯国际,2020,54.7933,0.032967


In [26]:
bloomberg_data_trans_path = Path(r"ESG_Data_transformed\Bloomberg_esg_rating_transformed.csv")
bloomberg_data.to_csv(bloomberg_data_trans_path, index=False, encoding="utf_8_sig")

# RUNLING

In [27]:
RUNLIN_data_path = Path("ESG_Data\润灵环球ESG评级19-23\ESG个股评级统计表150837633\ESG_StockRate.xlsx")
RUNLIN_data_raw = pd.read_excel(RUNLIN_data_path, sheet_name='模板')
RUNLIN_data_raw.drop(index=[0,1], inplace=True)
RUNLIN_data_raw.reset_index(drop=True, inplace=True)
RUNLIN_data_raw

,Symbol,ShortName,RateYear,IndexName,GICSIndustry1,GICSIndustryCode1,GICSIndustry2,GICSIndustryCode2,GICSIndustry3,GICSIndustryCode3,Escoring,Sscoring,Gscoring,ESGscoring,ESGrate
0,000869,张裕A,2019,中证800,必须消费品,30,视频、饮料和烟草,3020,饮料,302010,0,3.07,1.99,1.68,B
1,002304,洋河股份,2019,中证800,必须消费品,30,视频、饮料和烟草,3020,饮料,302010,2.45,3.01,3.37,2.94,BB
2,300012,华测检测,2020,中证800,工业,20,商业和专业服务,2020,专业服务,202020,0,0.5,3.15,1.22,CCC
3,600297,广汇汽车,2020,中证800,非必须消费品,25,零售,2550,专业零售,255040,0.6,1.8,2.78,1.73,B
4,601828,美凯龙,2020,中证800,非必须消费品,25,零售,2550,专业零售,255040,7.1,2.73,3.75,4.53,BBB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,000729,燕京啤酒,2023,中证800,日常消费品,30,食品、饮料与烟草,3020,饮料,302010,5.03,4.47,4.73,4.74,BBB
4136,002732,燕塘乳业,2023,NaN,日常消费品,30,食品、饮料与烟草,3020,饮料,302010,2.08,3.98,3.19,3.08,BB
4137,002304,洋河股份,2023,中证800,日常消费品,30,食品、饮料与烟草,3020,饮料,302010,6.1,4.73,5.35,5.39,BBB
4138,002461,珠江啤酒,2023,NaN,日常消费品,30,食品、饮料与烟草,3020,饮料,302010,2.4,2.7,3.85,2.98,BB


In [28]:
RUNLIN_data_trans = pd.DataFrame(columns=[
    'ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_RUNLIN'
])

for year in tqdm(range(2019, 2024)):
    data_year = RUNLIN_data_raw[RUNLIN_data_raw['RateYear'] == str(year)].reset_index(drop=True)
    data_year_select = data_year[["Symbol", "ShortName", "RateYear", "ESGscoring"]]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_score']
    data_year_select['esg_rank_norm_RUNLIN'] = 1 - data_year_select['esg_score'].rank() / data_year_select['esg_score'].rank().max()
    data_year_select['year'] = int(year)

    RUNLIN_data_trans = pd.concat([
        RUNLIN_data_trans,
        data_year_select[['ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_RUNLIN']]
    ])

RUNLIN_data_trans.sort_values(by=['ts_code', 'year'], inplace=True)
RUNLIN_data_trans.reset_index(drop=True, inplace=True)
RUNLIN_data_trans

  0%|          | 0/5 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_rank_norm_RUNLIN
0,000001,平安银行,2019,2.76,0.181250
1,000001,平安银行,2020,4.35,0.070253
2,000001,平安银行,2021,4.63,0.060777
3,000001,平安银行,2022,4.7,0.059637
4,000002,万科Ａ,2019,4.39,0.025000
...,...,...,...,...,...
4135,688737,中自科技,2023,3.32,0.325103
4136,688819,天能股份,2022,2.7,0.334123
4137,689009,九号公司,2022,2.5,0.388626
4138,836892,广咨国际,2023,2.04,0.659465


In [29]:
RUNLIN_data_trans_path = Path(r"ESG_Data_transformed\RUNLIN_esg_rating_transformed.csv")
RUNLIN_data_trans.to_csv(RUNLIN_data_trans_path, index=False, encoding="utf_8_sig")

# Allied Wave

In [30]:
allied_data_path = Path("ESG_Data\盟浪\盟浪2014-2023.xlsx")
allied_data_raw = pd.read_excel(allied_data_path, sheet_name='Sheet1')
allied_data_raw

,股票代码,年份,盟浪ESG评级
0,1,2014,A
1,1,2015,A+
2,1,2016,BBB+
3,1,2017,A-
4,1,2018,A-
...,...,...,...
53405,873833,2019,NaN
53406,873833,2020,NaN
53407,873833,2021,NaN
53408,873833,2022,NaN


In [31]:
allied_data_raw['盟浪ESG评级'].unique()

array(['A', 'A+', 'BBB+', 'A-', 'AA-', nan, 'B+', 'BB', 'B', 'CCC',
       'BBB-', 'BB+', 'BB-', 'B-', 'BBB', 'AA+', 'AA', 'CC'], dtype=object)

In [32]:
len(allied_data_raw['盟浪ESG评级'].unique())

18

In [33]:
allied_data_trans = pd.DataFrame(columns=[
    'ts_code', 'year', 'esg_score', 'esg_rank_norm_allied'
])

for year in tqdm(range(2014, 2024)):
    data_year = allied_data_raw[allied_data_raw['年份'] == year].reset_index(drop=True)
    data_year_select = data_year[["股票代码", "年份", "盟浪ESG评级"]]
    data_year_select.columns = ['ts_code', 'year', 'esg_score']
    data_year_select['esg_score'] = data_year_select['esg_score'].map(
        {
            "AA+":1, "AA":2, "AA-":3,
            "A+":4, "A":5, "A-":6,
            "BBB+":7, "BBB":8, "BBB-":9,
            "BB+":10, "BB":11, "BB-":12,
            "B+":13, "B":14, "B-":15,
            "CCC":16, "CC":17
        }
    )
    data_year_select['esg_rank_norm_allied'] = data_year_select['esg_score'].rank() / data_year_select['esg_score'].rank().max()

    allied_data_trans = pd.concat([
        allied_data_trans, 
        data_year_select[['ts_code', 'year', 'esg_score', 'esg_rank_norm_allied']]
    ])

allied_data_trans.sort_values(by=['ts_code', 'year'], inplace=True)
allied_data_trans.reset_index(drop=True, inplace=True)
allied_data_trans

  0%|          | 0/10 [00:00<?, ?it/s]

,ts_code,year,esg_score,esg_rank_norm_allied
0,1,2014,5.0,0.156794
1,1,2015,4.0,0.084977
2,1,2016,7.0,0.162516
3,1,2017,6.0,0.107855
4,1,2018,6.0,0.201245
...,...,...,...,...
53405,873833,2019,NaN,NaN
53406,873833,2020,NaN,NaN
53407,873833,2021,NaN,NaN
53408,873833,2022,NaN,NaN


In [34]:
allied_data_trans_path = Path(r"ESG_Data_transformed\Allied_Wave_esg_rating_transformed.csv")
allied_data_trans.to_csv(allied_data_trans_path, index=False, encoding="utf_8_sig")

# HEXUN

In [35]:
HEXUN_data_path = Path(r"ESG_Data/4-和讯社会责任报告评分（10-20年含细分项）/2010-2020年上市公司社会责任评分全部数据（和讯网）.xlsx")
hexun_data_raw = pd.read_excel(
    HEXUN_data_path, sheet_name="Sheet2"
)

In [36]:
hexun_data_raw.head(3)

,股票名称,股票代码,年份,总得分,等级,股东责任,员工责任,供应商、客户和消费者权益责任,环境责任,社会责任,...,环境治理得分,环保意识得分,环境管理体系认证得分,环保投入金额得分,排污种类数得分,节约能源种类数得分,社会责任得分,贡献价值得分,所得税占利润总额比得分,公益捐赠金额得分
0,兖州煤业,600188,2010,85.77,A,19.90,14.16,14,30.0,7.71,...,30.0,4.0,5.0,7.0,7.0,7.0,7.71,7.71,4.71,3.0
1,青岛啤酒,600600,2010,78.88,B,17.35,9.69,19,15.0,17.84,...,15.0,2.0,3.0,0.0,5.0,5.0,17.84,17.84,9.34,8.5
2,中国神华,601088,2010,78.67,B,19.44,12.50,11,27.0,8.73,...,27.0,4.0,2.0,7.0,7.0,7.0,8.73,8.73,3.73,5.0


In [37]:
hexun_data_trans = pd.DataFrame(
    columns=['ts_code', 'name', 'year', 'esg_score', 'esg_rank_norm_hexun']
)

for year in tqdm(range(2010, 2021)):
    data_year = hexun_data_raw[hexun_data_raw['年份'] == year].reset_index(drop=True)
    data_year_select = data_year[['股票代码', '股票名称', '年份', '总得分']]
    data_year_select.columns = ['ts_code', 'name', 'year', 'esg_score']
    data_year_select['esg_rank_norm_hexun'] = 1 - data_year_select['esg_score'].rank() / data_year_select['esg_score'].rank().max()

    hexun_data_trans = pd.concat([
        hexun_data_trans,
        data_year_select
    ])

hexun_data_trans.reset_index(drop=True, inplace=True)
hexun_data_trans.sort_values(by=['ts_code', 'year'], inplace=True)
hexun_data_trans

  0%|          | 0/11 [00:00<?, ?it/s]

,ts_code,name,year,esg_score,esg_rank_norm_hexun
111,1,平安银行,2010,61.93,0.046444
2563,1,平安银行,2011,66.44,0.064825
5278,1,平安银行,2012,66.53,0.079733
8073,1,平安银行,2013,71.28,0.055697
11050,1,平安银行,2014,77.70,0.002847
...,...,...,...,...,...
35954,688819,天能股份,2020,21.87,0.463498
25433,688981,中芯国际,2017,11.90,0.887045
28319,688981,中芯国际,2018,19.65,0.597783
32755,688981,中芯国际,2019,16.90,0.693002


In [38]:
hexun_data_path_trans = Path("ESG_Data_transformed/hexun_esg_rating_transformed.csv")
hexun_data_trans.to_csv(hexun_data_path_trans, index=False, encoding="utf_8_sig")

# Combine

In [16]:
dir_path = Path(r"ESG_Data_transformed")
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
csv_files.sort()
csv_files

['Allied_Wave_esg_rating_transformed.csv',
 'Bloomberg_esg_rating_transformed.csv',
 'CNRDS_esg_rating_transformed.csv',
 'FTSE_esg_rating_transformed.csv',
 'MSCI_esg_rating_transformed.csv',
 'RUNLIN_esg_rating_transformed.csv',
 'WIND_esg_rating_transformed.csv',
 'hexun_esg_rating_transformed.csv',
 'huazheng_esg_rating_transformed.csv',
 'syntao_esg_rating_transformed.csv']

In [17]:
ts_code_lst = []
year_lst = []

for filepath in csv_files:
    df = pd.read_csv(
        f"ESG_Data_transformed/{filepath}"
    )
    try:
        code_lst_df = df["ts_code"].apply(lambda x: int(x)).to_list()
        year_df = df['year'].apply(lambda x: int(x)).to_list()
    except:
        code_lst_df = df["ts_code"].apply(
            lambda x: int(x.split(".")[0])
        )
        year_df = df['year'].apply(lambda x: int(x)).to_list()

    ts_code_lst.extend(code_lst_df)
    year_lst.extend(year_df)


ts_code_lst = list(set(ts_code_lst))
year_lst = list(set(year_lst))
ts_code_lst.sort()
year_lst.sort()
len(ts_code_lst), len(year_lst)

(5706, 18)

In [18]:
def append_code_year(df_raw, ts_code_lst, year_lst):
    df = df_raw.copy()
    try:
        df['ts_code'] = df['ts_code'].apply(
            lambda x: int(x.split(".")[0])
        )
        df['year'] = df['year'].apply(lambda x:int(x))
    except:
        df['ts_code'] = df["ts_code"].apply(lambda x: int(x))
        df['year'] = df['year'].apply(lambda x:int(x))
    
    for code in ts_code_lst:
        for year in year_lst:
            if not ((df['ts_code']==code) & (df['year']==year)).sum():
                append_df_dict = {}
                for col in df.columns:
                    if col == "year":
                        append_df_dict[col] = [year]
                    elif col == "ts_code":
                        append_df_dict[col] = [code]
                    else:
                        append_df_dict[col] = [np.nan]

                df = pd.concat([
                    df, pd.DataFrame(append_df_dict)
                ])
    
    df.sort_values(by=['ts_code', "year"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [19]:
total_esg_df = pd.DataFrame()

for filepath in tqdm(csv_files, desc="combining"):
    file_df = pd.read_csv(f"ESG_Data_transformed/{filepath}")
    file_df_full = file_df.pipe(append_code_year, ts_code_lst, year_lst)
    del file_df

    select_col_lst = []
    for col in file_df_full.columns:
        if len(total_esg_df.columns) == 0:
            if col in ["year", "ts_code", "name"] or "industry" in col.lower() or "rank_norm" in col.lower():
                select_col_lst.append(col)
        else:
            if "industry" in col.lower() or "rank_norm" in col.lower():
                select_col_lst.append(col)
    
    total_esg_df = pd.concat([
        total_esg_df, file_df_full[select_col_lst]
    ], axis=1)

total_esg_df.to_csv("total_esg_data.csv", index=False, encoding="utf_8_sig")

combining:   0%|          | 0/10 [00:00<?, ?it/s]

# Compute divergence

**处理方法**：
1. 将不同的ESG评级转变为排名，并将其归一化至[0, 1]区间
2. 数据源之间两两配对，计算标准差作为该配对的分歧 i.e. 极差绝对值除以根号2 $\frac{|r_{i,t,1}-r_{i,t,2}|}{\sqrt{2}}$
3. 对每个公司每年的所有分歧取均值，作为该公司该年的ESG分歧指标

In [13]:
total_esg_df = pd.read_csv(
    "total_esg_data.csv"
)
total_esg_df.dropna(
    how='all', 
    subset=[col for col in total_esg_df.columns if "rank" in col],
    inplace=True
)
total_esg_df.reset_index(drop=True, inplace=True)
total_esg_df

,ts_code,year,esg_rank_norm_allied,esg_rank_norm_bloomberg,esg_rank_norm_CNRDS,esg_rank_norm_ftse,esg_rank_norm_MSCI,esg_rank_norm_RUNLIN,esg_rank_norm_WIND,CSRC_industry,WIND_industry,esg_rank_norm_hexun,esg_rank_norm_huazheng,esg_rank_norm_syntao
0,1.0,2007.0,NaN,0.801237,0.578591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2008.0,NaN,0.342187,0.690150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2009.0,NaN,0.281471,0.712924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007687,NaN
3,1.0,2010.0,NaN,0.275000,0.266205,NaN,NaN,NaN,NaN,NaN,NaN,0.046444,0.009353,NaN
4,1.0,2011.0,NaN,0.401639,0.509509,NaN,NaN,NaN,NaN,NaN,NaN,0.064825,0.010138,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59325,900948.0,2017.0,NaN,NaN,NaN,NaN,0.015094,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59326,900948.0,2018.0,NaN,NaN,NaN,NaN,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59327,900948.0,2019.0,NaN,NaN,NaN,NaN,0.036207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59328,900948.0,2020.0,NaN,NaN,NaN,NaN,0.009146,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
total_esg_df.columns

Index(['ts_code', 'year', 'esg_rank_norm_allied', 'esg_rank_norm_bloomberg',
       'esg_rank_norm_CNRDS', 'esg_rank_norm_ftse', 'esg_rank_norm_MSCI',
       'esg_rank_norm_RUNLIN', 'esg_rank_norm_WIND', 'CSRC_industry',
       'WIND_industry', 'esg_rank_norm_hexun', 'esg_rank_norm_huazheng',
       'esg_rank_norm_syntao'],
      dtype='object')

In [15]:
# rank_cols = [col for col in total_esg_df.columns if "rank" in col]

rank_cols = [
    "huazheng", "WIND", "syntao", "bloomberg", "ftse", "hexun", "RUNLIN"
]

for i in range(len(rank_cols)):
    rank_cols[i] = f"esg_rank_norm_{rank_cols[i]}"

for i in range(len(rank_cols)):
    for j in range(i+1, len(rank_cols)):
        data_name1 = rank_cols[i].split("_")[-1]
        data_name2 = rank_cols[j].split("_")[-1]
        col_name = f"std_{data_name1}_{data_name2}"
        total_esg_df[col_name] = total_esg_df[[rank_cols[i], rank_cols[j]]].std(axis=1)

total_esg_df.head()

,ts_code,year,esg_rank_norm_allied,esg_rank_norm_bloomberg,esg_rank_norm_CNRDS,esg_rank_norm_ftse,esg_rank_norm_MSCI,esg_rank_norm_RUNLIN,esg_rank_norm_WIND,CSRC_industry,...,std_syntao_bloomberg,std_syntao_ftse,std_syntao_hexun,std_syntao_RUNLIN,std_bloomberg_ftse,std_bloomberg_hexun,std_bloomberg_RUNLIN,std_ftse_hexun,std_ftse_RUNLIN,std_hexun_RUNLIN
0,1.0,2007.0,NaN,0.801237,0.578591,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2008.0,NaN,0.342187,0.690150,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2009.0,NaN,0.281471,0.712924,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2010.0,NaN,0.275000,0.266205,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.161614,NaN,NaN,NaN,NaN
4,1.0,2011.0,NaN,0.401639,0.509509,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.238164,NaN,NaN,NaN,NaN


In [16]:
total_esg_df['ESG_RANK_MEAN'] = total_esg_df[[col for col in total_esg_df.columns if "rank" in col]].mean(axis=1)
total_esg_df['ESG_RANK_STD'] = total_esg_df[[col for col in total_esg_df.columns if "std" in col]].mean(axis=1)

In [18]:
total_esg_df = total_esg_df[['ts_code', 'year', 'ESG_RANK_MEAN', 'ESG_RANK_STD']]
total_esg_df.columns = ['stkcd', 'year', 'ESG_RANK_MEAN', 'ESG_RANK_STD']
total_esg_df.to_csv("total_esg_std_data_selected.csv", index=False, encoding="utf_8_sig")